In [2]:
cd /home/haitaowu/baidu/DANet-paddle

/home/haitaowu/baidu/DANet-paddle


In [3]:
!which python

/home/haitaowu/anaconda3/bin/python


# 数据预处理
unzip解压数据集<br><br>
运行createTrainIdLabelImgs.py创建训练gt<br><br>
运行global_config.py检查文件结构；运行dataset_preprocess.py生成数据集列表



In [3]:
# 解压数据集中附带的官方处理代码
# !unzip -nq -d work/paddle/ data/cityscapesscripts.zip 
# 解压数据集中的gtFine
!unzip -nq -d data/gtFine/ data/gtFine_train.zip
!unzip -nq -d data/gtFine/ data/gtFine_val.zip
!unzip -nq -d data/gtFine/ data/gtFine_test.zip
# 解压数据集中的leftImg8bit
!unzip -nq -d data/leftImg8bit/ data/leftImg8bit_train.zip
!unzip -nq -d data/leftImg8bit/ data/leftImg8bit_val.zip
!unzip -nq -d data/leftImg8bit/ data/leftImg8bit_test.zip

^C
^C
^C
^C
^C


In [4]:
# 查看工作区文件夹
!tree -d work/
# 查看数据文件夹
!tree -d data/

work/
├── paddle
│   ├── cityscapesscripts
│   │   ├── annotation
│   │   │   ├── icons
│   │   │   └── __pycache__
│   │   ├── evaluation
│   │   ├── helpers
│   │   │   └── __pycache__
│   │   ├── preparation
│   │   │   └── __pycache__
│   │   └── viewer
│   │       └── icons
│   └── danet
│       ├── dataset
│       │   └── __pycache__
│       ├── my_utils
│       │   └── __pycache__
│       └── networks
│           └── __pycache__
└── result
    ├── txt_net
    └── txt_prepare

22 directories
data/

0 directories


In [ ]:
# 修改标签
!python work/paddle/cityscapesscripts/preparation/createTrainIdLabelImgs.py

Processing 3230 annotation files
Progress: 20.309597523219814 %   

In [7]:
# 文件结构检查
!python work/paddle/danet/my_utils/global_config.py
# 生成数据集列表
!python work/paddle/danet/dataset/dataset_preprocess.py

pass: /home/haitaowu/baidu/DANet-paddle/work/result
pass: /home/haitaowu/baidu/DANet-paddle/work/result/txt_prepare
pass: /home/haitaowu/baidu/DANet-paddle/work/result/txt_net
pass: /home/haitaowu/baidu/DANet-paddle/data
pass: /home/haitaowu/baidu/DANet-paddle/data/leftImg8bit
pass: /home/haitaowu/baidu/DANet-paddle/data/pre
pass: /home/haitaowu/baidu/DANet-paddle/data/infer
pass: /home/haitaowu/baidu/DANet-paddle/data/gtFine
train statistic...
Traceback (most recent call last):
  File "work/paddle/danet/dataset/dataset_preprocess.py", line 45, in <module>
    get_setCSV()
  File "work/paddle/danet/dataset/dataset_preprocess.py", line 22, in get_setCSV
    citys = os.listdir(dataImg_path)
FileNotFoundError: [Errno 2] No such file or directory: '/home/haitaowu/baidu/DANet-paddle/data/leftImg8bit/train'


# 网络训练
net_train.py训练模型，模型保存到work/result/***danet.pdparams***，训练曲线保存到work/result/txt_net/**train_record.png**<br><br>
net_test.py测试模型，混淆矩阵及miou保存到work/result/txt_net/**cfm_danet.csv**、work/result/txt_net/**test_mIoU.csv**<br><br>
net_infer.py模型推理<br><br>
<br>
最终精度miou 0.40

In [12]:
!which python

/opt/conda/envs/python35-paddle120-env/bin/python


In [ ]:

# !python work/paddle/danet/mytrain.py
#7+1+1
!python work/paddle/danet/net_train.py

dataset load...
model load...
W1016 20:08:32.317104  9047 device_context.cc:252] Please NOTE: device: 0, CUDA Capability: 70, Driver API Version: 11.0, Runtime API Version: 9.0
W1016 20:08:32.321872  9047 device_context.cc:260] device: 0, cuDNN Version: 7.6.
/home/aistudio/work/result/danet reload...
start train...
epoch 1/240
train: batch  54/198: acc 0.7295, loss 0.9156 [#############_____________________________________]27.3% 0.003996085;  

In [ ]:
!python work/paddle/danet/net_test.py

epoch7: 0.145057

In [ ]:
# !python work/paddle/danet/net_infer.py


* 数据预处理<br>
	* 使用Cityscapes-gtFine数据集，以8：2比例划分train、val（test）数据集<br>
	* train过程768*768随机crop，val、test过程768*768中心crop<br>
	* 自定义trainid为20类<br>
* 网络构建：<br>
	* 使用resnet50作为backbone，各block输出通道数由[64, 128, 256, 512]改为[32, 64, 128, 256]<br>
	* CAM、PAM，input_chs由2048改为1024，inter_chs由input_chs//4改为512<br>
	* 输出结果包括CAM、PAM、CAM+PAM3部分<br>
* train、val过程<br>
	* batch size=8<br>
	* epoch = 30<br>
	* optimizer<br>
		* Momentum，momentum=0.9，l2_decay=0.0001<br>
	* learning rate<br>
		* 由1e-4线性warmup至1e-1，2epoch；<br>
		* 由1e-1按(1e-1-1e-4)*((1-iter/total_iter)**0.9)+1e-4的polynomial_decay方法进行变化，30epoch（其中iter表示单个batch过程，1epoch表示所有batch过程，polynomial_decay的前两个epoch的学习率由warmup替换）<br>
	* loss<br>
		* softmax_with_cross_entropy<br>
		* 对CAM、PAM、CAM+PAM以0.3、0.3、0.4的比例进行加权<br>
	* 保存val accuracy最高、val loss最低2个模型<br>
	* 输出train、val过程的accuracy、loss曲线<br>
* test过程<br>
	* 计算并输出混淆矩阵、各类iou、miou、fwiou等指标<br>
* infer过程<br>
	* 对图像进行推理预测


最终结果：best val acc 0.8627，miou 0.4063，fwiou 0.7784<br>
<br>
由于对网络进行了一些魔改，加上只进行了2次30epoch的训练，最终结果相较于论文有一定差距<br>
<br>
因此，本复现结果只是对danet网络认识提供一种参考，希望大家可以一起交流学习<br>